In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
pd.set_option('display.max_colwidth', 500)
import time
import requests
import random

In [2]:
df = pd.read_csv('US_SHARES150422.csv')
df.head()

,Name,Ticker,SEDOL,ISIN,Exchange
0,1-800-FLOWERS.COM INC. CL A,FLWS,2444123,US68243Q1067,NASDAQ
1,1ST SOURCE CORP.,SRCE,2341848,US3369011032,NASDAQ
2,2U INC,TWOU,BKWBZZ0,US90214J1016,NASDAQ
3,3D SYSTEMS CORP,DDD,2889768,US88554D2053,New York Stock Exchange Inc.
4,3M CO,MMM,2595708,US88579Y1010,New York Stock Exchange Inc.


In [3]:
data = df['Ticker']
data.head()

0    FLWS
1    SRCE
2    TWOU
3     DDD
4     MMM
Name: Ticker, dtype: object

In [4]:
data.tail()

772     XPO
773     XYL
774    YHOO
775    YELP
776    ZNGA
Name: Ticker, dtype: object

In [5]:
data.info()

<class 'pandas.core.series.Series'>
RangeIndex: 777 entries, 0 to 776
Series name: Ticker
Non-Null Count  Dtype 
--------------  ----- 
777 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


In [6]:
def comp_name_bc(tick):
    if x%2 == 0:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0',}
    else:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36',}
    page = requests.get("https://www.barchart.com/stocks/quotes/" + tick, headers = headers)
    if page.status_code > 299:
        print("Error ", page.status_code, "for", tick)
    soup = bs(page.content)
    Company_unedited = str(soup.find(class_ ='symbol'))
    result = re.search('">(.*)<', Company_unedited)
    if result:
        return result.group(1), tick
    else:
        print("Error ", page.status_code, "for", tick, "name not found in", Company_unedited)
    #time.sleep(1)

In [7]:
def comp_name_yf(tick):
    if x%2 == 0:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0',}
    else:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36',}
    page = requests.get("https://finance.yahoo.com/quote/" + tick, headers = headers)
    if page.status_code > 299:
        print("Error ", page.status_code, "for", tick)
    soup = bs(page.content)
    Company_unedited = str(soup.find_all(class_='D(ib) Fz(18px)'))
    result = re.search('">(.*)<', Company_unedited)
    if result:
        return result.group(1)
    else:
        print("Error ", page.status_code, "for", tick, "name not found in", Company_unedited)
    #time.sleep(1)

In [8]:
def comp_name_final(tick):
    company_final_yf = comp_name_yf(tick)
    if company_final_yf:
        return company_final_yf
    else:
        return comp_name_bc(tick)

In [9]:
import regex as re
import numpy as np
import time as time
comp_names_array = ['' for _ in range(len(df))] # above line
for x in range(0, len(data)): #len(data)
    if x%30 == 0:
        print('progress', x/len(data) * 100, '%')
    tickn = data[x]
    company_name_new = comp_name_final(tickn)
    comp_names_array[x] = company_name_new #no need to append because of above line
print(comp_names_array)

progress 0.0 %
Error  200 for ANH name not found in []
progress 3.861003861003861 %
Error  200 for ARRS name not found in []
Error  200 for ASNA name not found in []
Error  200 for ASCMA name not found in []
Error  200 for AST name not found in []
Error  200 for HOME name not found in []
Error  200 for ATHN name not found in []
progress 7.722007722007722 %
Error  200 for AT name not found in []
Error  200 for BDGE name not found in []
Error  200 for BPI name not found in []
Error  200 for BGG name not found in []
Error  200 for BLMT name not found in []
Error  200 for COG name not found in []
progress 11.583011583011583 %
Error  200 for CAI name not found in []
Error  200 for CCC name not found in []
Error  200 for CXP name not found in []
Error  200 for CHUBK name not found in []
progress 15.444015444015443 %
Error  200 for CVRS name not found in []
Error  200 for CSOD name not found in []
Error  200 for ICBK name not found in []
Error  200 for CVA name not found in []
Error  200 for 

In [10]:
df['NameUpd'] = comp_names_array

In [11]:
df.head()

,Name,Ticker,SEDOL,ISIN,Exchange,NameUpd
0,1-800-FLOWERS.COM INC. CL A,FLWS,2444123,US68243Q1067,NASDAQ,"1-800-FLOWERS.COM, Inc. (FLWS)"
1,1ST SOURCE CORP.,SRCE,2341848,US3369011032,NASDAQ,1st Source Corporation (SRCE)
2,2U INC,TWOU,BKWBZZ0,US90214J1016,NASDAQ,"2U, Inc. (TWOU)"
3,3D SYSTEMS CORP,DDD,2889768,US88554D2053,New York Stock Exchange Inc.,3D Systems Corporation (DDD)
4,3M CO,MMM,2595708,US88579Y1010,New York Stock Exchange Inc.,3M Company (MMM)


In [12]:
df.to_excel(excel_writer = "/Users/failul/USShares150422.xlsx")